In [1]:
import os
import sys
import h5py
import numpy as np
import pandas as pd

from pynwb import NWBFile, NWBHDF5IO
from pynwb.ophys import ImageSegmentation, PlaneSegmentation

In [2]:
pos_array = np.random.uniform(0, 1, size=(4, 3)).astype(float)

x = pos_array[:, 0]
y = pos_array[:, 1]
z = pos_array[:, 2]
t = np.arange(0, 4)

wlid = ['Name 1', 'Name 2', 'Name 3', 'Name 4']

old_file = 'old_file.nwb'
new_file = 'new_file.nwb'

In [3]:
# Open nwb file
io = NWBHDF5IO(old_file, mode="r")
nwbfile = io.read()

# Grab existing object's ImagingVolume
calc_imaging_vol = nwbfile.processing['CalciumActivity'].data_interfaces['TrackedNeurons'].plane_segmentations['TrackedNeuronROIs'].imaging_plane

# Remove existing object in file
nwbfile.processing['CalciumActivity'].data_interfaces.pop('TrackedNeurons')

# Create neuron track objects...
NeuroPALTracks = ImageSegmentation(
    name='TrackedNeurons',
)

trackIDs = PlaneSegmentation(
    name='TrackedNeuronROIs',
    description='Neuron centers as tracked throughout GCaMP video. Formatted as (x, y, z, t)',
    imaging_plane=calc_imaging_vol,
)

# ...add ROIs...
for eachNeuron in range(len(x)):
    trackIDs.add_roi(voxel_mask=[[x[eachNeuron], y[eachNeuron], z[eachNeuron], t[eachNeuron]]])

# ...add neuron IDs...
label = []
for eachNeuron in range(len(x)):
    label.append(wlid[eachNeuron])

trackIDs.add_column(
    name='TrackedNeuronIDs',
    description='Neuron ID labels for tracked neuron rois.',
    data=label,
    index=False,
)

print(f"Pre-export:\n{type(trackIDs.voxel_mask.data[0][0])}\n{trackIDs.voxel_mask.data}")

# ...save to parent object
NeuroPALTracks.add_plane_segmentation(trackIDs)

# Save objects to cached nwbfile.
nwbfile.processing['CalciumActivity'].add(NeuroPALTracks)

# Save changes to file.
nwbfile.set_modified()

with NWBHDF5IO(new_file, mode='w') as export_io:
    export_io.export(src_io=io, nwbfile=nwbfile)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'old_file.nwb', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
test_io = NWBHDF5IO(new_file, mode="r")
test_file = test_io.read()
pos_data = test_file.processing['CalciumActivity'].data_interfaces['TrackedNeurons'].plane_segmentations['TrackedNeuronROIs'].voxel_mask.data
print(f"Post-export:\n{type(pos_data[0][0])}\n{pos_data[:]}")